In [1]:
import pandas_datareader.data as web
import datetime

In [2]:
start =datetime.datetime(2000,1,1)
end = datetime.datetime(2023,9,30)

In [3]:
# Get data
df = web.DataReader('GOOGL','stooq',start,end)
df.dropna(inplace=True)
df.sort_index(inplace=True)
df

,Open,High,Low,Close,Volume
Date,,,,,
2004-08-19,2.50000,2.6015,2.39900,2.50850,894076000.0
2004-08-20,2.52525,2.7270,2.51250,2.70775,457144000.0
2004-08-23,2.76875,2.8370,2.72625,2.73500,365488000.0
2004-08-24,2.78100,2.7900,2.58925,2.62175,305252000.0
2004-08-25,2.62400,2.7000,2.59700,2.65000,183956000.0
...,...,...,...,...,...
2023-09-25,129.83000,131.1700,128.96000,131.11000,20094643.0
2023-09-26,129.77000,130.3645,127.22000,128.56500,25718704.0
2023-09-27,128.57000,130.8950,128.57000,130.54000,22746452.0


In [4]:
# predict in 10 days
pre_days =10
df['label'] = df['Close'].shift(-pre_days)
df

,Open,High,Low,Close,Volume,label
Date,,,,,,
2004-08-19,2.50000,2.6015,2.39900,2.50850,894076000.0,2.53775
2004-08-20,2.52525,2.7270,2.51250,2.70775,457144000.0,2.50025
2004-08-23,2.76875,2.8370,2.72625,2.73500,365488000.0,2.53950
2004-08-24,2.78100,2.7900,2.58925,2.62175,305252000.0,2.55750
2004-08-25,2.62400,2.7000,2.59700,2.65000,183956000.0,2.55775
...,...,...,...,...,...,...
2023-09-25,129.83000,131.1700,128.96000,131.11000,20094643.0,NaN
2023-09-26,129.77000,130.3645,127.22000,128.56500,25718704.0,NaN
2023-09-27,128.57000,130.8950,128.57000,130.54000,22746452.0,NaN


In [5]:
# data standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sca_X = scaler.fit_transform(df.iloc[:,:-1])
sca_X

D:\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


array([[-1.02104717, -1.0181845 , -1.02349222, -1.02064401,  5.14743076],
       [-1.02037223, -1.01486759, -1.02042475, -1.01531999,  2.23843076],
       [-1.01386334, -1.01196033, -1.0146479 , -1.01459186,  1.62820468],
       ...,
       [ 2.34887511,  2.372558  ,  2.38643035,  2.40039955, -0.6536954 ],
       [ 2.38282292,  2.43612113,  2.41940231,  2.44769453, -0.65524879],
       [ 2.47477607,  2.45594331,  2.43480724,  2.40895006, -0.59967853]])

In [6]:
# memory days
mem_his_days = 15

In [7]:
# building featutres

from collections import deque
deq = deque(maxlen=mem_his_days)

X = []
for i in sca_X:
    deq.append(list(i))
    if len(deq)==mem_his_days:
        X.append(list(deq))
        
X_lately = X[-pre_days:]
X=X[:-pre_days]
print(len(X))
print(len(X_lately))

y = df['label'].values[mem_his_days-1:-pre_days]
print(len(y))

4787
10
4787


In [8]:
import numpy as np

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

In [11]:
# Build LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [12]:
model = Sequential()
model.add(LSTM(10,input_shape=X.shape[1:],activation='relu',return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(10,activation='relu',return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(10,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1))

model.compile(optimizer='adam',
             loss='mse',
             metrics=['mape'])

model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

Epoch 1/50
135/135 [==============================] - 4s 9ms/step - loss: 2609.6531 - mape: 92.8464 - val_loss: 171.3901 - val_mape: 36.0337
Epoch 2/50
135/135 [==============================] - 1s 7ms/step - loss: 378.6451 - mape: 31.1367 - val_loss: 54.7268 - val_mape: 13.1278
Epoch 3/50
135/135 [==============================] - 1s 7ms/step - loss: 251.9998 - mape: 25.2206 - val_loss: 48.7448 - val_mape: 12.9695
Epoch 4/50
135/135 [==============================] - 1s 7ms/step - loss: 249.2244 - mape: 24.0020 - val_loss: 103.0899 - val_mape: 15.5960
Epoch 5/50
135/135 [==============================] - 1s 8ms/step - loss: 236.6054 - mape: 23.5841 - val_loss: 22.7318 - val_mape: 9.9480
Epoch 6/50
135/135 [==============================] - 1s 8ms/step - loss: 220.7125 - mape: 22.4716 - val_loss: 80.7712 - val_mape: 10.2681
Epoch 7/50
135/135 [==============================] - 1s 7ms/step - loss: 222.2602 - mape: 21.4704 - val_loss: 100.3347 - val_mape: 11.3548
Epoch 8/50
135/135 [====